# Setup

We begin by importing what we need, and loading a simulation. We rescale the time for convenience so that a simulation time of 1 corresponds to one orbit of the outermost planet.

In [1]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "../binaries/trappist.bin"
sim = rebound.Simulation.from_file(filename)
sim.t = 0
ss.rescale_time(sim, sim.particles[-1].P)

We first add a `FrameRecorder` to write frames for the movie. We have to choose how much simulation time will correspond to one second in the movie. Here we choose for each unit of time (outer planet orbits) to correspond to two seconds of movie time. We also record all the transits and conjunctions.

In [2]:
frames = ss.FrameRecorder(sim, time_per_sec=0.5)
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y, targets=range(1,8))
conjunctions = ss.EventRecorder(sim, lambda sim, i: np.sin(sim.particles[i].theta - sim.particles[i+1].theta), targets=range(1,7))

As an overview, the approach is for the `FrameRecorder` to record events at all the moments when we need to write a frame, and we'll later write a function to generate each of those frames. This makes it easier to parallelize, which is nice since generating individual plots with matplotlib is quite slow.

With that in mind, we can also add any metadata to the `FrameRecorder` that we later want to use to write the movie frames. This will be saved in each of the frame events, e.g.:

In [3]:
frames.color = [None, 'cyan','dodgerblue','hotpink','crimson','chocolate','forestgreen','yellowgreen']

We'll make a simple movie where we only display the outer 3 planets. We do that by changing `frames.plotparticles` to store the particles we want displayed.

In [4]:
frames.plotparticles = [5,6,7]
sim.integrate(tmax=2) # integrate for two outer planet orbits

We now have events for all the frames in the movie in `frames.events`, all the transits in `transits.events` and conjunctions in `conjunctions.events`.

# Writing the movie frames

We now need to write a function that writes movie frames for each of the events. Each frame event stores basic information for the frame in a dictionary, notably a filename to a binary that stores the simulation at the appropriate time for the frame, as well as any metadata we have added before/between calls to sim.integrate (e.g. the `color` field we added above). We can see this in the first frame event:

In [5]:
frames.events[0]

{'color': [None,
  'cyan',
  'dodgerblue',
  'hotpink',
  'crimson',
  'chocolate',
  'forestgreen',
  'yellowgreen'],
 'elapsed_time': 0.03537080119148124,
 'filename': 'tmp/binaries/frame0.bin',
 'fps': 30,
 'frame_ctr': 0,
 'plotparticles': [5, 6, 7],
 'target': None,
 'time': 0.016666662112231917,
 'time_per_sec': 0.5}

We now need to write a function that will write a movie frame from this information. Below is a simple one we could use, whee we take the variables we need, and leave the remaining dictionary entries unspecified in kwargs. We'll be writing many frames (to the tmp/pngs/ directory), so it's important to close the figures after each one.

In [6]:
import matplotlib.pyplot as plt
def write_png(params):
    sim = rebound.Simulation.from_file(params['filename'])
    fig = rebound.OrbitPlot(sim, figsize=(8,8), plotparticles=params['plotparticles'])
    fig.axes[0].axis('off') # turn off axes
    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(params['frame_ctr']))
    plt.close(fig)

We now write all the frames using our function. Making matplotlib plots is not optimized for efficiency, so it's quite slow. We therefore generate them in parallel.

In [7]:
%%time
from subprocess import call
call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
res = pool.map(write_png, frames.events)

CPU times: user 24.9 ms, sys: 38.3 ms, total: 63.2 ms
Wall time: 5.3 s


# Outputting a movie file

You now have all the frames for the movie in `systemsounds/jupyter_examples/tmp/pngs`, and can stitch them together into a movie using your favorite software. For some ideas and options, see [MovieEditingSoftware.ipynb](MovieEditingSoftware.ipynb)

# More advanced example

Everyone will want to do something slightly differently, so rather than building up a complicated framework with many options, the idea is that people can contribute examples to this repository that others can draw on to make their own custom movies. We're looking forward to seeing what people come up with! For a more complicated example see [TRAPPIST1.ipynb](TRAPPIST1.ipynb)